# VERSION ETUDIANTS 
# TP MASI Corrélation et Convolution : du signal à l'image


Le but de ce TP est de manipuler un peu les notions théoriques liées à la corrélation et à la convolution que nous avons vues en cours, pour un signal 1D pour commencer, et ensuite pour une image 2D. Attention, ici on va travailler sur la convolution et la corrélation non pas sous la forme analogique, mais sous en numérique.


In [ ]:
### Les imports ###

import numpy as np
import scipy as sp
from scipy import signal
import skimage as sk
import skimage.io
import matplotlib.pyplot as plt
# Matplotlib in inline mode
%matplotlib inline

# Partie 1 : avec des fonctions toutes faites

Ci-dessous, vous trouverez deux signaux : x et y

In [ ]:
####################### defining time vector  #######################
tmax = 15 # max time (in sec)
tmin = 0 # min time (does not mean anything but has to be defined)
fe = 250 # sampling frequency (in Hz)
Te = 1/fe # sampling period (fe points per second)
t = np.arange(tmin,tmax,Te)
#####################################################################

# Let's define two signals
x = sp.signal.chirp(t, 0.25, t.max(), 2)
y = np.exp(-t)


plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,x,label=r'$x(t)$')
plt.xlim(t.min(),t.max())
plt.ylim(ymax=1.25)
plt.legend(loc='upper right')
plt.subplot(122)
plt.plot(t,y,label=r'$y(t)$')
plt.xlim(t.min(),t.max())
plt.ylim(ymax=1.25)
plt.legend(loc='upper right')
plt.legend(loc='upper right')
plt.show()

## A vous de jouer !

<font color="blue"> Quelle est la relation entre corrélation et convolution ?</font>


Jouez avec les fonctions toutes faites sp.signal.correlate et sp.signal.convolve, appliquées aux signaux x et y définis juste au dessus

#### La corrélation déjà...

In [ ]:
t_corr = np.arange(-tmax+Te,tmax,Te)
G_xy = #... A compléter
plt.plot(t_corr,G_xy,label=r'$\Gamma_{xy}(\tau)$')
plt.xlim(t_corr.min(),t_corr.max())
plt.legend(loc='best',fontsize=14)
plt.show()

#### ...puis la convolution...

In [1]:
Conv_xy = #... A compléter
plt.plot(t_corr,Conv_xy,label=r'$(x \ast y)(\tau)$')
plt.xlim(t_corr.min(),t_corr.max())
plt.legend(loc='best',fontsize=14)
plt.show()

SyntaxError: invalid syntax (3344334950.py, line 1)

#### ...et enfin : retrouvez la relation entre corrélation et convolution !

Indice : vous pouvez regarder la documentation de [`np.flip`](https://numpy.org/doc/stable/reference/generated/numpy.flip.html) ou l'effet de `a[::-1]` pour un array `a` donné

In [ ]:
#... A compléter...

# Partie 2 : Application de la corrélation pour trouver l'origine d'un séisme.
L'idée est ici de voir comment utiliser la corrélation afin de déterminer l'origine d'un séisme. 

### Un peu de contexte... 
Vous êtes des chercheurs du CNRS spécialisés en tremblement de terre. Un tremblement de terre vient de se produire quelque part dans le monde, et vous voulez savoir où. Vous vous connectez donc à vos 3 sondes éparpillées sur la Terre entière afin de récupérer les données des sismographes, et à l'aide de triangulation, déterminer l'origine du séisme en question. 

Attention, vos sondes ne sont pas toutes de qualité égale, vous recevez des signaux bruts, pensez à regarder les notices de vos sondes pour pouvoir calculer l'heure de réception des ondes !


**Sonde 1** : Paris. C'est la sonde la plus performante et précise. Elle enregistre une valeur toutes les 0.5 secondes. De plus, elle est dans un endroit protégé et très peu soumise aux interférences extérieures.

**Sonde 2** : Sao Paulo. Cette sonde est moins performante. Elle enregistre une valeur par seconde. Elle n'est pas très bien isolée, Bernardo, le technicien en charge de sa maintenance, avait malheureusement sa fille de 3 ans qui faisait de la corde à sauter avec lui au moment du tremblement de terre...

**Sonde 3** : Sidney. C'est la pire des sondes. Elle enregistre une valeur toutes les 2 secondes, et en plus le métro passe juste en dessous, provoquant beaucoup de tremblements parasites.

### Bases de sismologie

On va considérer 2 types d'ondes : les ondes P (horizontales, les plus rapides) et les ondes S (qui arrivent en 2e). 

Pour une station, on peut écrire :

*Temps d’arrivée de l’onde P* : $t_P = t_0 + (d/v_P)$

*Temps d’arrivée de l’onde S* : $t_S = t_0 + (d/v_S)$

où $d$ est la distance à l'épicentre, $t_0$ le moment du séisme, $v_P$ la vitesse des ondes $P$ et $v_S$ la vitesse des ondes $S$.

Problème : on ne connaît pas $t_0$ car on a uniquement les relevés de nos 3 sondes ! 0n va alors isoler la distance $d$ en faisant la différence entre les deux relations précédentes : $t_S - t_P = d × ( 1/v_S -1/v_P)$.

On admettra que $(1/v_S-1/v_P) = 1/8$s/km.

Au final, on obtient pour une station l'estimation de la distance à l'épicentre suivante : 

**$d = 8 × (t_S - t_P)$**

$d$ est en km, $t_S - t_P$ en secondes.

On a donc uniquement besoin de connaître le temps qui s'est écoulé entre l'arrivée des deux types d'ondes ! Cela nous donne la distance entre l'épicentre et la sonde : l'épicentre se situe sur un cercle de rayon $d$ centré sur la sonde. En faisant ce calcul pour 3 sondes, on obtient un unique point au croisement de ces 3 cercles : l'origine de notre séisme !
<img src="seisme.png">

On considèrera que la distance $d$ est environ égale à la distance à vol d'oiseau entre la sonde et l'épicentre. Il y a 6 candidats pour l'épicentre :

| Candidat | Distance Sonde 1 | Distance Sonde 2 | Distance Sonde 3 |
|----------|------------------|------------------|------------------|
| New York | 5800 km          | 7700 km          | 16000 km
| Tokyo    | 9700 km          | 18500 km         | 7800 km
| Moscou   | 2500 km          | 11800 km         | 14000 km
| Impfondo         | 5800 km          | 7700 km          |  14000 km
| Ninghua    | 9700 km          | 18500 km          | 7600 km
| Quelque part au large du Groenland         | 2500 km          | 11800 km          | 16000 km

Dans le dossier "signaux", vous avez les signaux enregistrés par les 3 sondes.

### <font color="red"> * * * * * * * * * * * * * * * * * * * * * * * Attention : ne pas toucher aux cellules suivantes * * * * * * * * * * * * * * * * * * * * * * </font>

Il s'agit de la définition des templates des ondes P et S qui vous serviront pour la suite. 
P1 et S1 correspondent aux signaux P et S avec la résolution de la sonde 1, P2 et S2 pour la sonde 2 et P3 et S3 pour la sonde 3.

In [ ]:
# Template des signaux P1, P2 et P3
tmax = 15 # max time (in sec)
tmin = 0 # min time (does not mean anything but has to be defined)
fe = 250 # sampling frequency (in Hz)
Te = 1/fe # sampling period (fe points per second)
t = np.arange(tmin,tmax,Te)
A_ref = 1
t0 = 1.2
f0 = 1
P1 = A_ref*np.exp(-t/t0)*np.sin(2*np.pi*f0*t)
fe2 = 250/2 # sampling frequency (in Hz)
Te2 = 1/fe2 # sampling period (fe points per second)
t2 = np.arange(tmin,tmax,Te2)
P2 = A_ref*np.exp(-t2/t0)*np.sin(2*np.pi*f0*t2)
fe3 = 250/4 # sampling frequency (in Hz)
Te3 = 1/fe3 # sampling period (fe points per second)
t3 = np.arange(tmin,tmax,Te3)
P3 = A_ref*np.exp(-t3/t0)*np.sin(2*np.pi*f0*t3)
# ----------------------------------
# -------- plot that stuff ---------
# ----------------------------------
plt.figure(figsize=(15,6))
plt.subplot(131)
plt.plot(t,P1)
plt.xlim((t.min(),t.max()))
plt.xlabel('Time (in seconds)')
plt.title('P1')
plt.subplot(132)
plt.plot(t2, P2)
plt.xlim((t.min(),t.max()))
plt.xlabel('Time (in seconds)')
plt.title('P2')
plt.subplot(133)
plt.plot(t3, P3)
plt.xlim((t.min(),t.max()))
plt.xlabel('Time (in seconds)')
plt.title('P3')
plt.show()

In [ ]:
# Template des signaux S1, S2 et S3
tmax = 15 # max time (in sec)
tmin = 0 # min time (does not mean anything but has to be defined)
fe = 250 # sampling frequency (in Hz)
Te = 1/fe # sampling period (fe points per second)
t = np.arange(tmin,tmax,Te)
A_ref = 1
t0 = 1
f0 = 1
S1 = A_ref*np.exp(-0.5*t/t0)*np.sin(5*np.pi*f0*t)
fe2 = 250/2 # sampling frequency (in Hz)
Te2 = 1/fe2 # sampling period (fe points per second)
t2 = np.arange(tmin,tmax,Te2)
S2 = A_ref*np.exp(-0.5*t2/t0)*np.sin(5*np.pi*f0*t2)
fe3 = 250/4 # sampling frequency (in Hz)
Te3 = 1/fe3 # sampling period (fe points per second)
t3 = np.arange(tmin,tmax,Te3)
S3 = A_ref*np.exp(-0.5*t3/t0)*np.sin(5*np.pi*f0*t3)
# ----------------------------------
# -------- plot that stuff ---------
# ----------------------------------
plt.figure(figsize=(15,6))
plt.subplot(131)
plt.plot(t,S1)
plt.xlim((t.min(),t.max()))
plt.xlabel('Time (in seconds)')
plt.title('S1')
plt.subplot(132)
plt.plot(t2,S2)
plt.xlim((t.min(),t.max()))
plt.xlabel('Time (in seconds)')
plt.title('S2')
plt.subplot(133)
plt.plot(t3,S3)
plt.xlim((t.min(),t.max()))
plt.xlabel('Time (in seconds)')
plt.title('S3')
plt.show()

### <font color="red"> * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * </font>

### A vous de jouer ! 

<font color="blue"> 
Calculez $d_1$, $d_2$ et $d_3$ à partir des signaux contenus dans le dossier "signaux". Retrouvez l'origine du séisme parmi les 6 candidats. </font>

Faites très attention à la résolution des signaux : avec la corrélation, récupérez vos résultats en terme de nombre de points entre les deux signaux (servez-vous de [`np.argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) ), puis convertissez en secondes en fonction de la précision de la sonde. Pour Sao Paulo, 1 point = 1 seconde, facile ! Ce n'est pas le cas pour les deux autres sondes...

In [ ]:
### Chargement de l'enregistrement de la sonde 1 Paris
Sonde_1 = np.load("signaux/sonde1.npy")
plt.plot(Sonde_1,label='Sonde 1 - Paris')
plt.xlim(0,Sonde_1.size)
plt.legend(loc='best',fontsize=14)
plt.show()

In [ ]:
# Recherche du retard du premier signal P1 dans l'enregistrement 1
PP1 = #... A compléter

In [ ]:
# Recherche du retard du signal S1 dans l'enregistrement 1
SS1 = #... A compléter

In [ ]:
# Calcul de ts-tp en nombre de points
#... A compléter

In [ ]:
# Attention, on a un décalage en points. Or d'apres la notice de la sonde, 1 seconde = combien de points ?  
# modifiez la valeur de ts-tp pour l'avoir en secondes... 
#... A compléter
# Ce qui nous donne pour d :
#... A compléter

##### Appliquons la même procédure pour la sonde 2 :

In [ ]:
### Chargement de l'enregistrement de la sonde 2 Sao Paulo
Sonde_2 = np.load("signaux/sonde2.npy")
plt.plot(Sonde_2,label='Sonde 2 - Sao Paulo')
plt.xlim(0,Sonde_2.size)
plt.legend(loc='best',fontsize=14)
plt.show()

In [ ]:
PP2 = #... A compléter

In [ ]:
SS2 = #... A compléter

In [ ]:
# Calcul de ts-tp en nombre de points
#... A compléter

In [ ]:
# Attention, on a un décalage en points. Or d'apres la notice de la sonde, 1 seconde = combien de points ?  
# modifiez la valeur de ts-tp pour l'avoir en secondes... 
#... A compléter
# Ce qui nous donne pour d :
#... A compléter

##### Et enfin la sonde 3

In [ ]:
### Chargement de l'enregistrement de la sonde 3 Sidney
Sonde_3 = np.load("signaux/sonde3.npy")
plt.plot(Sonde_3,label='Sonde 3 - Sydney')
plt.xlim(0,Sonde_3.size)
plt.legend(loc='best',fontsize=14)
plt.show()

In [ ]:
PP3 = #... A compléter

In [ ]:
SS3 = #... A compléter

In [ ]:
# Calcul de ts-tp en nombre de points
#... A compléter

In [ ]:
# Attention, on a un décalage en points. Or d'apres la notice de la sonde, 1 seconde = combien de points ?  
# modifiez la valeur de ts-tp pour l'avoir en secondes... 
#... A compléter
# Ce qui nous donne pour d :
#... A compléter

##### Quel candidat correspond aux 3 distances ?

Réponse :

# Partie 3 : passons à l'image maintenant !

Jusqu'à présent, on a joué avec des signaux. C'est bien joli tout ça, mais ce qui nous intéresse maintenant c'est de s'amuser avec... des images ! 

On va prendre une image (au hasard, évidemment), par exemple celle-ci :<br>
<img src="image_chat.jpg">

La convolution en 2D présente quelques subtilités avec la version 1D. En particulier, en 1D, on ne s'est pas forcément intéressés à la problématique des bords.



Ci-dessous un exemple illustré sur une "image" (signal en 2D).

![Alt Text](https://miro.medium.com/max/1400/1*O06nY1U7zoP4vE5AZEnxKA.gif)



Dans cette illustration, nous avons "l'image" en bleu (notre signal f), le noyau de convolution (aussi appelé filtre) en orange (notre signal g), le résultat final en vert. Chaque carré de ces "images" est un "pixel".

Dans cet exemple, le résultat a la même taille que l'image initiale. Chaque valeur du résultat correspond à la convolution entre l'image initiale et le noyau de convolution. Qu'est ce que ça veut dire ? La "nouvelle valeur" d'un pixel correspond à la somme de la multiplication terme à terme de deux matrices : celle du noyau et une sous-matrice de la même taille que le noyau, centrée sur le pixel étudié.

Prenons le pixel central de l'exemple ci-dessus : le pixel "70". En centrant le noyau dessus, on va avoir : 

<font color="blue"> 121</font>x<font color="orange">0</font>+<font color="blue">54</font>x<font color="orange">(-1)</font>+<font color="blue"> 84</font>x<font color="orange">0</font>+<font color="blue"> 99</font>x<font color="orange">(-1)</font>+<font color="blue">70</font>x<font color="orange">5</font>+<font color="blue"> 129</font>x<font color="orange">(-1)</font>+<font color="blue"> 57</font>x<font color="orange">0</font>+<font color="blue">115</font>x<font color="orange">(-1)</font>+<font color="blue"> 69</font>x<font color="orange">0</font> = <font color="green">-47</font>

Que se passe-t-il aux bords ? 
La bordure blanche autour de l'image bleue représente le "padding". Il s'agit d'un ajout que l'on fait aux bordures pour que le noyau ne sorte pas de l'image. En effet, le noyau centré sur un pixel du bord va déborder et retourner une erreur. Pour éviter cela, on rajoute artificiellement autant de lignes et de colonnes que de débordement : c'est le "padding". Ici, ce "padding" est fait avec des 0, mais il peut également être fait avec la valeur moyenne de l'image, avec une duplication miroir des lignes et des colonnes etc. Si l'on reprend le calcul à la main sur le premier pixel de l'image, le "60", on a donc : 

<font color="blue"> 0</font>x<font color="orange">0</font>+<font color="blue">0</font>x<font color="orange">(-1)</font>+<font color="blue"> 0</font>x<font color="orange">0</font>+<font color="blue"> 0</font>x<font color="orange">(-1)</font>+<font color="blue">60</font>x<font color="orange">5</font>+<font color="blue"> 113</font>x<font color="orange">(-1)</font>+<font color="blue"> 0</font>x<font color="orange">0</font>+<font color="blue">73</font>x<font color="orange">(-1)</font>+<font color="blue"> 121</font>x<font color="orange">0</font> = <font color="green">114</font>

<font color="blue"> **Exercice :** Recodez vous même la fonction de convolution entre l'image du chat et un noyau de taille $n\times n$.</font>

#### Etape 1 : convolution avec le noyau ci-dessus !
2 parties : sur la matrice bleue (ça vous permet de vérifier votre fonction), puis sur le chat !

Dans cette partie, on vous demande de recoder la fonction de convolution et de l'appliquer sur l'image-exemple bleue du dessus (vérifiez que les valeurs finales que vous obtenez correspondent bien aux valeurs de l'image verte !), puis sur l'image du chat. 

In [ ]:
################### NE PAS TOUCHER ##################
# Cette cellule correspond à l'image bleue et au noyau orange
image = np.array([[60,113,56,139,85],
                  [73,121,54,84,128],
                  [131,99,70,129,127],
                  [80,57,115,69,134],
                  [104,126,123,95,130]])
noyau = np.zeros((3,3))
noyau[1,:]=-1
noyau[:,1]=-1
noyau[1,1]=5
######################################################

In [ ]:
def ma_conv_img(input_image, input_noyau):
    #Commencez par le padding
    image_pad = #... A compléter
    #Puis faites glisser le noyau sur l'image et calculez le résultat de convolution
    for i in range(#... A compléter):
        for j in range(#... A compléter):
            #... A compléter
            #... A compléter
            #... A compléter
    return(#... A compléter)

ma_conv_img(image, noyau)

##### Image du chat
On va maintenant appliquer cette fonction de convolution à l'image "image_chat.jpg". Une image couleur, c'est en fait 3 grilles de pixels : une pour le canal rouge, une pour le canal vert et une pour le canal bleu (les fameuses images "RGB"). Les valeurs d'un pixel dans chaque canal permettent de générer la couleur voulue à partir du rouge/vert/bleu qui sont l'équivalent des couleurs primaires en peinture. Observez les différences entre les canaux (en particulier le mur au fond à droite).

In [ ]:
image_cat = sk.io.imread('image_chat.jpg')
plt.figure(figsize=(15,10))
plt.subplot(231)
plt.imshow(image_cat[:,:,0], cmap ='gray')
plt.title('Rouge')
plt.subplot(232)
plt.imshow(image_cat[:,:,1], cmap ='gray')
plt.title('Vert')
plt.subplot(233)
plt.imshow(image_cat[:,:,2], cmap ='gray')
plt.title('Bleu')
plt.subplot(235)
plt.imshow(image_cat)
plt.title('Couleur')
plt.show()

Pour la suite, on va juste utiliser le canal vert pour l'exemple.

<font color="blue"> Appliquez votre fonction de convolution avec le noyau précédent sur le canal vert. Que remarquez-vous ?</font>

In [ ]:
img_chat_convol = #... A compléter

Ici, ce n'est pas un exemple très intéressant. Regardons ce qu'il se passe si on modifie le noyau ...
#### Petite vérification avant de continuer...
On se souvient que lors de la convolution, le 2e signal (ici notre noyau), doit subir un petit "flip". Or nous n'avons vu qu'un noyau symétrique. Que se passe-t-il avec un noyau assymétrique ? Nous allons vérifier ça tout de suite à l'aide de "noyau2".

In [ ]:
noyau2 = np.zeros((3,3))
noyau2[0:3,2]=-1
noyau2[0:3,0]=1
noyau2

<font color="blue"> Appliquez votre fonction de convolution avec noyau2. Comparez au résultat de sp.signal.convolve avec le même noyau.</font>
Regardez surtout les zones noires et blanches des oreilles des chiens : sont-elles au même endroit ?

In [ ]:
avec_convol = #... A compléter
avec_scipy = #... A compléter
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.imshow(avec_convol, cmap = 'gray')
plt.title('ma convolution')
plt.subplot(122)
plt.imshow(avec_scipy, cmap = 'gray')
plt.title('Scipy')
plt.show()

#### Etape 2 : on va maintenant utiliser votre fonction pour faire un filtre moyenneur de taille $3 \times 3$...

Pour faire un filtre moyenneur, il faut faire un noyau...moyenneur ! Rappel, la moyenne est la somme de termes divisée par le nombre de termes. Ca ne ressemble pas un peu à la convolution ?

Testez dans un premier temps sur l'image-jouet ("l'image bleue" du début, celle de l'animation) pour vérifier vos valeurs, puis sur l'image du chat. Que remarquez-vous sur l'image du chat ? 


In [ ]:
noyau_moy = #... A compléter

##### Et le chat ?

In [ ]:
conv_moy_cat = #... A compléter

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(121)
plt.imshow(image_cat[:,:,1], cmap ='gray')
plt.title('Canal vert')
plt.subplot(122)
plt.imshow(conv_moy_cat, cmap ='gray')
plt.title('Résultat de la convolution 3x3')
plt.show()

#### ... puis de taille $5 \times 5$ uniquement sur le chat ...

In [ ]:
#... A compléter

In [ ]:
#... A compléter

In [ ]:
#... A compléter

#### ... puis de taille $9 \times 9$ uniquement sur le chat ...

In [ ]:
#... A compléter

In [ ]:
#... A compléter

In [ ]:
#... A compléter

#### Que remarquez-vous ? 

#### Pouvez vous faire de même pour un filtre médian de taille $3 \times 3$? 